## گام یکم: تبدیل داده ها به دیتافریم

In [ ]:
import pandas as pd

samples = pd.read_csv('data/rocksamples.csv')

## گام دوم: پاکسازی داده ها

In [ ]:
# مرحله اول: تماشای داده ها - توصیف
samples.info()
samples.head(10)
samples.tail()
samples.describe()

In [ ]:
# مرحله اول: تماشای داده ها - نمودار
#بر پایه متپلات لایب
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(samples['Mission'], samples['Weight (g)'])
ax.set_xlabel('Mission')
ax.set_ylabel('Weight (g)')
plt.show()

In [ ]:
# بر پایه ای.پی.آی پانداز
samples.plot(kind='scatter', x='Pristine (%)', y='Weight (g)')

In [ ]:
# پیش از مرحله بعدی یک کپی از دیتافریم بگیرید
samples_m = samples.copy()

In [ ]:
# مرحله دوم: برخورد با داده های تهی و سطرهای تکراری
# جایگزینی سلول های تهی با یک مقدار معین
mean = samples_m['Weight (g)'].mean()
median = samples_m['Weight (g)'].median()
mode = samples_m['Weight (g)'].mode()[0]
samples_m['Weight (g)'].fillna(mean, inplace=True) # or use 'median'/'mode'
# حذف سطرهای حاوی حتی یک سلول تهی
samples_m.dropna(inplace=True)
# حذف سطرهای تکراری
print(samples_m.duplicated())
samples_m.drop_duplicates(inplace=True)
samples_m.info()

In [ ]:
# مرحله سوم و چهارم: مناسب کردن داده ها برای تحلیل های بعدی
samples_m['Weight (g)'] = samples_m['Weight (g)'].apply(lambda x : x * 0.001)
samples_m.head()

In [ ]:
# مرحله پنجم: اصلاح عنوان سرستون ها
samples_m.rename(columns={'Weight (g)' : 'Weight (kg)'}, inplace=True)
samples_m.info()

In [ ]:
# مرحله ششم: داده های پرت - تشخیص
import seaborn as sb
sb.boxplot(x=samples_m['Weight (kg)'])
sb.boxplot(x=samples_m['Pristine (%)'])

In [ ]:
# مرحله ششم: داده های پرت - برخورد

# حذف داده های پرت وزن
# q1_w = samples_m['Weight (kg)'].quantile(0.25)
# q3_w = samples_m['Weight (kg)'].quantile(0.75)
# iqr_w = q3_w - q1_w
# print(iqr_w)

# for i in samples_m.index:
#     if samples_m.loc[i, 'Weight (kg)'] > (q3_w + 1.5*iqr_w):
#         samples_m.drop(i,inplace=True)

# حذف داده های پرت درصد تازگی
for i in samples_m.index:
    if samples_m.loc[i, 'Pristine (%)'] > 100:
        samples_m.drop(i,inplace=True)
        
samples_m.info()

## گام سوم: تحلیل داده ها

In [ ]:
# ایجاد یک دیتافریم با ساختار تازه برای واریز نتایج تحلیل
missions = pd.DataFrame()
missions

In [ ]:
# استخراج دسته ها از ستون
missions['Mission'] = samples_m['Mission'].unique()
missions

In [ ]:
# جمع داده های یک ستون بر اساس گروه داده های آن
total_samples_weight = samples_m.groupby('Mission')['Weight (kg)'].sum()
total_samples_weight

In [ ]:
# ادغام یک سری در یک دیتافریم
missions = pd.merge(missions, total_samples_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

In [ ]:
# تفاضل داده ها در یک ستون
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

In [ ]:
# افزودن ستون (سری) به دیتافریم موجود
missions['Lunar module (LM)'] = ['Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)']
missions['LM mass (kg)'] = [15103, 15235, 15264, 16430, 16445, 16456]
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = ['Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)']
missions['CM mass (kg)'] = [5560, 5609, 5758, 5875, 5840, 5960]
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

In [ ]:
# جمع داده های ستون ها در دیتافریم
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions